In [59]:
import json
import time
import hashlib

- An index: Some non-negative integer indicating which block number it is in the chain.
- A set of records. Let's call these transactions.
- A timestamp. Let's call this timestamp.
- A hash of the current block. Let's call this property hash.
- The backwards linking hash to the previous block. Let's call this previous_hash.
- A number-used-once or nonce; as a property called nonce. 

In [60]:
class Block():

    def __init__(self,
                 index=0,
                 transactions=[],
                 timestamp=0,
                 current_hash="",
                 pervious_hash="",
                 nonce=0,):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.current_hash = current_hash
        self.pervious_hash = pervious_hash
        self.nonce = nonce
        
    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

In [66]:
class Blockchain:

    def __init__(self):
        self.unconfirmed_transactions = []
        self.difficulty = 2
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    # todo define get_last_block(self) -> Block, hint: consider an index to
    # the last element in your chain property.
    #
    def get_last_block(self):
        return self.chain[-1]

    # todo define proof_of_work(self, block:Block) -> str,
    # hint consider something similar to:
    # computed_hash = how do we get the hash from a Block object?
    # while not computed_hash.startswith('0' * Blockchain.difficulty):
    #         block.nonce += 1
    #         computed_hash = block.compute_hash()
    # Don't forget to return the hash.
    #

    def proof_of_work(self, block):
        computed_hash = block.compute_hash()
        while not computed_hash.startswith("0" * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash

    # todo define is_valid_proof(self, block:Block, block_hash:str) -> bool
    # hint, we want to return true if block_hash.startswith('0' * Blockchain.difficulty)
    # AND if block_hash is in fact the hash of our block (maybe use block.compute_hash())
    #

    def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith("0" * self.difficulty)) and (block_hash == block.compute_hash())

    # todo define add_block(self, block:Block, proof:str) -> bool
    # hints, we probably want to get the previous hash from get_last_block()
    # Check if the previous hash matches the previous hash in the block argument.
    # Make use of is_valid_proof for block and proof to check if someone is trying to do something sneaky.
    # for both of the above, let's return False early and return control the calling function if the block is not valid.
    # If we make it this far, we're in the clear. So let's add the hash to our block, append it to the chain,
    # and return True.
    #

    def add_block(self, block, proof):

        last_block = self.get_last_block()

        if last_block.current_hash != block.pervious_hash:
            return False

        if not self.is_valid_proof(block, proof):
            return False

        block.current_hash = proof
        self.chain.append(block)

        return True

    # todo, define add_new_transaction(self, transaction:dict) -> None
    # Hint how do we append to a list? How do we do this for a property of our object (self). Ease one line function.    
    #

    def add_new_transaction(self, transaction):
        self.unconfirmed_transactions.append(transaction)

    # todo, define mine(self) -> int:
    # Hints: do we need to continue if our mempool is empty? Maybe return -1 if so. 
    # consider retrieving the last block to a local variable make life easy. 
    # Let's make sure we use our Block constructor to create a new block with all the transactions we want to mine 
    # (all of them are fine)
    # hint: Block(index=last_block.index + 1,
    #                    transactions=self.unconfirmed_transactions,
    #                    timestamp=time.time(),
    #                    previous_hash=last_block.hash)
    #

    def mine(self):
        if len(self.unconfirmed_transactions) < 1:
            return -1

        last_block = self.get_last_block()
        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=int(time.time()),
                          pervious_hash=last_block.current_hash,
                          )
        current_hash = self.proof_of_work(new_block)


    # Finally, let's be sure to use our handy proof_of_work function, add_block function, and to remember to reset our
    # unconfirmed_transactions (our mem-pool), before returning our new block index;

        if not self.add_block(new_block, current_hash):
            return -1
        
        new_block.current_hash = current_hash
        self.unconfirmed_transactions = []
        return new_block.index


In [67]:
tx_1 = {
   "addr_from": "3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",
   "addr_to": "1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",   
   "amount": 99.00
}
tx_2 = {
   "addr_from": "3J98t1WpEZ73CNmQviecunyirn8RhWNLy",
   "addr_to": "1BvBMSEYstWetqTFn5AuimdGFg7xJaNVN2",   
   "amount": 100.00
}
tx_3 = {
   "addr_from": "0",
   "addr_to": "1",   
   "amount": 1000000.00
}

In [68]:
bitcoins = Blockchain()
bitcoins.chain

In [69]:
bitcoins.add_new_transaction([tx_1, tx_2, tx_3])
bitcoins.mine()

1

In [70]:
bitcoins.chain

[<__main__.Block at 0x7f2926d39e80>, <__main__.Block at 0x7f2926d39f40>]